<a href="https://colab.research.google.com/github/rileeeeeeee/Rileystff/blob/main/Final_Project_Passcode_Generator_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#Passcode Generator
#Generates a passcode between with length 13 and 99,999 characters
#Riley Ward | Last edited 1/2/23
!apt install libmagic-dev
!pip install pycryptodomex
#imports needed
import time
import random
import os
import magic
from os import urandom
from hashlib import md5
from Cryptodome.Cipher import AES
from pathlib import Path
#lists of the characters the program will randomly choose from to generate the password
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
symbols = ['!', '#', '$', '%', '&', '(', ')', '*', '+', '_', '~', '-', '|', '^']
password = "99-4010"
#if the password is correct the user goes on but if its not it loops back and asks again in 10 seconds, it will infinitely loop until/unless the user inputs the correct password
def verify_password():
  while True: 
    global password_check
    password_check = input("Enter the password: ")
    if password_check == password:
      print("Password accepted.")
      break
    else:
      print("Password incorrect, now you go in time out.")
      time.sleep(10)
#get the user to enter the length of their passcode, has to be above 12 if it is it generates the passcode from the lists above and prints it to the user, if its not above 12 and an integer it loops and asks again
def generate_passcode():
  while True:
    character_num = input("Enter the number of characters above 12 and below 100,000 for your passcode length. ")
    try:
      character_num = int(character_num)
      if character_num > 12 and character_num < 100000 and character_num:
        break
    except ValueError:
      pass

  passcode_array = []
  character_possibilities = letters + numbers + symbols

  for char in range(character_num):
    passcode_array += random.choice(character_possibilities)

  global passcode
  passcode = ""
  for char in passcode_array:
    passcode += char

  print("Your generated passcode is: \n" + passcode)
#Asking what website the password is for 
def title_passcode():
  global title
  title = input("What is the site you want to tie this password to? ")
#Writes the passcode into the file alongside the user entered site 
def write_passcode():
  file1 = open(r"Passfile.txt", "a") 
  file1.write(title + ": " + passcode + "\n")
  file1.close()
#Opens the decrypted file (Passcode)
def read_passcode():
  file1 = open('Passfile.txt', 'r')
  print(file1.read())

#Found online
#Encrypts the file block by block in 16 byte chunks and appends them all together into the encrypted file
def encrypt(in_file, out_file, password, key_length=32):
    bs = AES.block_size #16 bytes
    salt = urandom(bs) #return a string of random bytes
    key, iv = derive_key_and_iv(password, salt, key_length, bs)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    out_file.write(salt)
    finished = False

    while not finished:
        chunk = in_file.read(1024 * bs) 
        if len(chunk) == 0 or len(chunk) % bs != 0:#final block/chunk is padded before encryption
            padding_length = (bs - len(chunk) % bs) or bs
            chunk += str.encode(padding_length * chr(padding_length))
            finished = True
        out_file.write(cipher.encrypt(chunk))
#Found online
#Basically the opposite of the above function, decrypts the encrypted file block by block and appends the blocks together into the decrypted file
def decrypt(in_file, out_file, password, key_length=32):
    bs = AES.block_size
    salt = in_file.read(bs)
    key, iv = derive_key_and_iv(password, salt, key_length, bs)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    next_chunk = ''
    finished = False
    while not finished:
        chunk, next_chunk = next_chunk, cipher.decrypt(in_file.read(1024 * bs))
        if len(next_chunk) == 0:
            padding_length = chunk[-1]
            chunk = chunk[:-padding_length]
            finished = True 
        out_file.write(bytes(x for x in chunk)) 
#Found online
#Creates the key and initialization vector required for MODE_CBC encryption
def derive_key_and_iv(password, salt, key_length, iv_length): #derive key and IV from password and salt.
    d = d_i = b''
    while len(d) < key_length + iv_length:
        d_i = md5(d_i + str.encode(password) + salt).digest() #obtain the md5 hash value
        d += d_i
    return d[:key_length], d[key_length:key_length+iv_length]
#calling the functions
verify_password()
file_exists = os.path.exists("/tmp/Passfile-Encrypted.txt")
if file_exists == True: 
  with open('/tmp/Passfile-Encrypted.txt', 'rb') as in_file, open('Passfile.txt', 'wb') as out_file:
    decrypt(in_file, out_file, password)
generate_passcode()
title_passcode()
write_passcode()
#asking if user wants to open the file containing their saved passcodes
choice = input("Do you want to view all saved passcodes? (Y/N) ")
if choice == "Y":
  read_passcode()
else:
  print("Exiting now.")

choice_2 = input("Do you want to delete your saved passcodes file? (Y/N) ")
if choice_2 == "Y":
  #os.remove('/tmp/Passfile-Encrypted.txt') Getting error 'file not found' can't find a fix except using below command
  !rm /tmp/Passfile-Encrypted.txt > /dev/null 2>&1
else:
  with open('Passfile.txt', 'rb') as in_file, open('/tmp/Passfile-Encrypted.txt', 'wb') as out_file:
    encrypt(in_file, out_file, password_check)



    
  file_type = magic.from_file('Passfile.txt')
  print("File type of Decrypted file: " + file_type)
  file_type = magic.from_file('/tmp/Passfile-Encrypted.txt')
  print("File type of Encrypted file: " + file_type)
delete_file("Passfile.txt")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagic-dev is already the newest version (1:5.32-2ubuntu0.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Enter the password: 99-4010
Password accepted.
Enter the number of characters above 12 and below 100,000 for your passcode length. 15
Your generated passcode is: 
|IDcJkD54a7JN6N
What is the site you want to tie this password to? H
Do you want to view all saved passcodes? (Y/N) Y
::::;q: A1N(~+f5^kSjkd~
H: |IDcJkD54a7JN6N

Do you want to delete your saved passcodes file? (Y/N) Y


#Had the most trouble with saving the passcodes to a file and adding subsequent passwords to that file
#Biggest challenge was figuring out how to start the code after writing the password
#My dad helped me figure out how to format the for char in range section in the last 10 or so lines of code
#Proud, because it wasn't the easiest thing I could think of to do and I had to learn some new things on python through googling to write this
#I would add after the user successfully generates a passcode, they are asked if they would like to repeat the sequence and if they say yes it repeats, if they say no the program ends